## API 
http://dart.fss.or.kr/api/search.xml?auth=xxxxx 사용 

In [8]:
# DART Open API 를 사용하기 위해서는 인증키를 사용해야 된다 
import requests 
auth = 'fa2804dc433cff0900e8107d9c6afd00382f6fd9' 
url_temp = 'http://dart.fss.or.kr/api/search.xml?auth={auth}'
url = url.format(auth = auth)

r = requests.get(url)
r.text[:100]

'<?xml version="1.0" encoding="utf-8"?>\n<result>\n  <err_code>000</err_code>\n  <err_msg>정상</err_msg>\n '

## 기업 개황 API 
http://dart.fss.or.kr/api/company.json?auth=xxx&crp_cd=xxx 사용 

In [54]:
import json
from pandas.io.json import json_normalize 

url_temp = 'http://dart.fss.or.kr/api/company.json?auth={auth}&crp_cd={crp}'
url = url_temp.format(auth = auth , crp = '005930')
r = requests.get(url)

samsung = json_normalize(json.loads(r.content))


## 정보 검색 API 
http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003

- crp_cd: 공시대상회사의 종목코드(상장사:숫자 6자리) 또는 고유번호(기타법인:숫자 8자리)
- end_dt: 검색종료 접수일자(YYYYMMDD) : 없으면 당일
- start_dt: 검색시작 접수일자(YYYYMMDD) : 없으면 end_dt crp_cd가 없는 경우 검색기간은 3개월로 제한
- fin_rpt: 최종보고서만 검색여부(Y or N) 기본값 : N (정정이 있는 경우 최종정정만 검색)
- dsp_tp: 정기공시(A), 주요사항보고(B), 발행공시(C), 지분공시(D), 기타공시(E), 외부감사관련(F), 펀드공시(G), 자산유동화(H), 거래소공시(I), 공정위공시(J)
- bsn_tp: 정기공시(5), 주요사항보고(3), 발행공시(11), 지분공시(4), 기타공시(9), 외부감사관련(3), 펀드공시(3), 자산유동화(6), 거래소공시(6), 공정위공시(5)
- sort: 접수일자(date), 회사명(crp), 보고서명(rpt) 기본값 : date
- series: 오름차순(asc), 내림차순(desc) 기본값 : desc
- page_no: 페이지 번호(1~n) 기본값 : 1
- page_set: 페이지당 건수(1~100) 기본값 : 10, 최대값 : 100
- callback: 콜백함수명(JSONP용)


In [53]:
# 당일 접수 100건 
url_temp ="http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100"
url = url_temp.format(auth = auth)

r = requests.get(url)
news = json_normalize(json.loads(r.content) , 'list')
news = news[['crp_cd' , 'crp_cls' , 'crp_nm' , 'rmk' , 'rpt_nm']]
news.head(n=10)

,crp_cd,crp_cls,crp_nm,rmk,rpt_nm
0,016360,Y,삼성증권,,투자설명서(일괄신고)
1,005930,Y,삼성전자,,임원ㆍ주요주주특정증권등소유상황보고서
2,01010749,E,케이제이엠파트너즈제이차,,감사보고서 (2018.02)
3,01006357,E,디에스에프제일차,,감사보고서 (2018.02)
4,241820,K,피씨엘,,[기재정정]사업보고서 (2017.12)
5,030610,Y,교보증권,,증권발행실적보고서
6,005930,Y,삼성전자,,임원ㆍ주요주주특정증권등소유상황보고서
7,124500,K,아이티센,,주식등의대량보유상황보고서(약식)
8,237720,N,케이엠제약,넥,기업설명회(IR)개최
9,00160144,E,한국투자증권,,일괄신고추가서류(파생결합사채-주가연계파생결합사채)


In [52]:
# 임원ㆍ주요주주특정증권등소유상황보고서 회사들 찾아오는 function 만들기 
def ownership_delta() : 
    # STEP 1 : 새로운 공시 다운로드 
    auth = 'fa2804dc433cff0900e8107d9c6afd00382f6fd9' 
    url_temp ="http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100"
    url = url_temp.format(auth = auth)
    r = requests.get(url)
    news = json_normalize(json.loads(r.content) , 'list')
    news = news[['crp_cd' , 'crp_cls' , 'crp_nm' , 'rmk' , 'rpt_nm']]
    
    # STEP 2 : '임원ㆍ주요주주특정증권등소유상황보고서' 있는 경우만 
    import pandas as pd 
    df = pd.DataFrame([news.loc[x] for x in range(len(news)) if news['rpt_nm'][x] == '임원ㆍ주요주주특정증권등소유상황보고서'])
    return df

ownership_delta().head(n=5)

,crp_cd,crp_cls,crp_nm,rmk,rpt_nm
1,005930,Y,삼성전자,,임원ㆍ주요주주특정증권등소유상황보고서
6,005930,Y,삼성전자,,임원ㆍ주요주주특정증권등소유상황보고서
15,078600,K,대주전자재료,,임원ㆍ주요주주특정증권등소유상황보고서
17,005930,Y,삼성전자,,임원ㆍ주요주주특정증권등소유상황보고서
22,120240,K,대정화금,,임원ㆍ주요주주특정증권등소유상황보고서


In [51]:
# 단일판매ㆍ공급계약체결 회사들 찾아오는 function 만들기 
def bigsales() : 
    # STEP 1 : 새로운 공시 다운로드 
    auth = 'fa2804dc433cff0900e8107d9c6afd00382f6fd9' 
    url_temp ="http://dart.fss.or.kr/api/search.json?auth={auth}&page_set=100"
    url = url_temp.format(auth = auth)
    r = requests.get(url)
    news = json_normalize(json.loads(r.content) , 'list')
    news = news[['crp_cd' , 'crp_cls' , 'crp_nm' , 'rmk' , 'rpt_nm']]
    
    # STEP 2 : '임원ㆍ주요주주특정증권등소유상황보고서' 있는 경우만 
    import pandas as pd 
    df = pd.DataFrame([news.loc[x] for x in range(len(news)) if news['rpt_nm'][x] == '단일판매ㆍ공급계약체결'])
    return df

bigsales().head(n=5)

,crp_cd,crp_cls,crp_nm,rmk,rpt_nm
95,046390,K,삼화네트웍스,코,단일판매ㆍ공급계약체결


## 보고서 다운받기 : 재무제표 및 다른 데이타 크롤링
http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170515003806




In [84]:
# Example : 비아트론 
# 사업+반기+분기보고서 : 사업 A001 , 반기 A002 , 분기 A003 
code = '141000'
url_temp = 'http://dart.fss.or.kr/api/search.json?auth={auth}&crp_cd={code}&start_dt=19990101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003&page_set=100' 
url = url_temp.format(auth = auth , code = code)
r = requests.get(url)
docs = json_normalize(json.loads(r.content), 'list')
docs

,crp_cd,crp_cls,crp_nm,flr_nm,rcp_dt,rcp_no,rmk,rpt_nm
0,141000,K,비아트론,비아트론,20180518,20180518000333,,분기보고서 (2018.03)
1,141000,K,비아트론,비아트론,20180402,20180402004590,연,사업보고서 (2017.12)
2,141000,K,비아트론,비아트론,20171128,20171128000306,,분기보고서 (2017.09)
3,141000,K,비아트론,비아트론,20170822,20170822000291,,반기보고서 (2017.06)
4,141000,K,비아트론,비아트론,20170526,20170526000442,,분기보고서 (2017.03)
5,141000,K,비아트론,비아트론,20170331,20170331004782,연,사업보고서 (2016.12)
6,141000,K,비아트론,비아트론,20161114,20161114001666,,분기보고서 (2016.09)
7,141000,K,비아트론,비아트론,20160816,20160816000457,,반기보고서 (2016.06)
8,141000,K,비아트론,비아트론,20160516,20160516002795,,분기보고서 (2016.03)
9,141000,K,비아트론,비아트론,20160330,20160330000705,,사업보고서 (2015.12)


In [107]:
# 링크 뽑아내기 
for x in range(len(docs)): 
    url_tmpl = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}"
    url = url_tmpl.format(docs['rcp_no'][x])
    print (x, url)

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180518000333
1 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180402004590
2 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171128000306
3 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170822000291
4 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170526000442
5 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004782
6 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20161114001666
7 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160816000457
8 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160516002795
9 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20160330000705
10 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20151113000858
11 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150813000148
12 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150512002958
13 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20150331003249
14 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20141114000702
15 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20140814000026
16 http://dart.fss

### url 파일 포맷 
- pdf_link_tmpl = "http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
- excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
- ifrs_link_tmpl = "http://dart.fss.or.kr/pdf/download/ifrs.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"

In [161]:
# rcp_no와 dcm_no을 알면 다운로드 할 수 있음 

def get_report_attach_urls(rcp_no):
    pdf_link_tmpl = "http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no={rcp_no}&dcm_no={dcm_no}"
    excel_link_tmpl = "http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
    ifrs_link_tmpl = "http://dart.fss.or.kr/pdf/download/ifrs.do?rcp_no={rcp_no}&dcm_no={dcm_no}&lang=ko"
    
    attach_urls = []
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s' %(rcp_no)
    r = requests.get(url)
    
    start_str = "javascript: viewDoc\('" + rcp_no + "', '"
    end_str = "', null, null, null,"
    reg = re.compile(start_str + '(.*)' + end_str)
    m = reg.findall(r.text)
    dcm_no = m[0]

    attach_urls.append(pdf_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(excel_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    attach_urls.append(ifrs_link_tmpl.format(rcp_no=rcp_no, dcm_no= dcm_no))
    return attach_urls

In [214]:
get_report_attach_urls('20180518000333')

['http://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20180518000333&dcm_no=6185965',
 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20180518000333&dcm_no=6185965&lang=ko',
 'http://dart.fss.or.kr/pdf/download/ifrs.do?rcp_no=20180518000333&dcm_no=6185965&lang=ko']

In [282]:
# import os # file 이름 rename 해줄 때 필요 
# import os 해서 rename 을 해줄 수도 있음 
def download_file(url) :
    reg = re.compile('rcp_no=' + '(.*)' + '&dcm_no')
    file = reg.findall(url)[0][7:]
    local_filename = file + "_보고서" 
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [284]:
# 비아트론 2018년 1분기 보고서 다운로드 
a = get_report_attach_urls('20180518000333')[1]
b = download_file(a)
b

'8000333_보고서'

In [463]:
import pandas as pd 
import numpy as np
df = pd.read_excel(b , sheet_name= '연결 재무상태표', skiprows=5)
df

,,제 18 기 1분기말,제 17 기말
0,자산,,
1,유동자산,125371706190,113399274468
2,현금및현금성자산,32548578436,41676876041
3,유동금융자산,18735842829,21949439766
4,매출채권,16733738871,5763562608
5,유동확정계약자산,16379326652,38810004280
6,기타채권,507780295,1114985591
7,이연법인세자산,3703070,0
8,재고자산,37188644896,62304141
9,파생상품자산,0,36116616


In [464]:
df['제 18 기 1분기말'] = pd.to_numeric(df['제 18 기 1분기말'], errors='coerce')
df['제 17 기말'] = pd.to_numeric(df['제 17 기말'], errors='coerce')

In [466]:
df['change'] = df['제 18 기 1분기말']/df['제 17 기말'] -1 

In [467]:
df

,,제 18 기 1분기말,제 17 기말,change
0,자산,NaN,NaN,NaN
1,유동자산,1.253717e+11,1.133993e+11,0.105578
2,현금및현금성자산,3.254858e+10,4.167688e+10,-0.219025
3,유동금융자산,1.873584e+10,2.194944e+10,-0.146409
4,매출채권,1.673374e+10,5.763563e+09,1.903367
5,유동확정계약자산,1.637933e+10,3.881000e+10,-0.577961
6,기타채권,5.077803e+08,1.114986e+09,-0.544586
7,이연법인세자산,3.703070e+06,0.000000e+00,inf
8,재고자산,3.718864e+10,6.230414e+07,595.888815
9,파생상품자산,0.000000e+00,3.611662e+07,-1.000000
